<a href="https://colab.research.google.com/github/JCaballerot/Recommender-Systems/blob/main/SVD_Recommender/SVD_Collaborative_Filtering_Last_fm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> SVD Collaborative Filtering - Last.fm </font></h1>

---

**Índice**

- 1. Introducción
- 2. Carga y Filtrado de Datos
- 3. Análisis de Energía Acumulada para SVD
- 4. Creación del Modelo SVD
- 5. Generación de Recomendaciones
- 6. Validación
- 7. Conclusiones


## 1. Introducción

Este laboratorio aplica SVD con filtrado "long tail" a los datos de interacciones usuario-artista en Last.fm. Exploraremos la energía acumulada para determinar el número adecuado de factores y evaluaremos el modelo ocultando el 20% de los ítems por usuario para probar la capacidad predictiva del modelo.



Instalamos las librerías necesarias.

In [ ]:
# Instalar Surprise
!pip install scikit-surprise

# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse


## 2. Carga y Filtrado de Datos

Cargamos el dataset y aplicamos un filtro "long tail" para mejorar la calidad del análisis, manteniendo solo los artistas con al menos 50 escuchas. Este enfoque reduce el impacto de artistas menos populares y permite centrarse en recomendaciones más relevantes.



In [ ]:
from google.colab import files
files.upload()  # Sube tu archivo kaggle.json aquí

# Crear la carpeta .kaggle y mover el archivo
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d japarra27/lastfm-dataset
!unzip lastfm-dataset.zip


In [ ]:
# Cargar el dataset
data = pd.read_parquet("lastfm_union.parquet")[:1_000_000]

In [ ]:
# Contar las escuchas por artista
artist_listen_counts = data.groupby('artist_name').size().sort_values(ascending=False)

# Visualizar distribución long tail
plt.figure(figsize=(12, 6))
plt.bar(range(len(artist_listen_counts)), artist_listen_counts, color='lightblue')
plt.title('Distribución del Número de Escuchas por Artista (Long Tail)')
plt.xlabel('Artistas ordenados por popularidad')
plt.ylabel('Número de escuchas')
plt.ylim(1, 4000)
plt.show()


In [7]:
# Filtrar artistas con al menos 50 escuchas
min_listens_per_artist = 50
popular_artists = artist_listen_counts[artist_listen_counts >= min_listens_per_artist].index
data_filtered = data[data['artist_name'].isin(popular_artists)]

# Crear la matriz usuario-artista con el recuento de escuchas
user_artist_matrix = data_filtered.groupby(['user_id', 'artist_name']).size().unstack(fill_value=0)


In [ ]:
user_artist_matrix.head()

## 3. Análisis de Energía Acumulada para SVD


Para determinar el número óptimo de factores, usamos SVD sobre la matriz usuario-artista y calculamos la energía acumulada. El objetivo es cubrir al menos el 90% de la variabilidad para lograr una buena representación sin perder demasiada información.

In [20]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

import numpy as np

# Convertir la matriz usuario-artista a formato sparse matrix
user_artist_matrix_sparse = csr_matrix(user_artist_matrix, dtype=np.float32)

# Realizar la descomposición SVD
U, sigma, Vt = svds(user_artist_matrix_sparse, k = min(user_artist_matrix_sparse.shape) - 1)
sigma = np.flip(np.sort(sigma))  # Ordenar valores singulares de mayor a menor


In [22]:

# Calcular la energía acumulada
explained_variance_ratio = sigma**2 / np.sum(sigma**2)
explained_variance_cumsum = np.cumsum(explained_variance_ratio)


In [ ]:
# Graficar la energía acumulada
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance_cumsum) + 1), explained_variance_cumsum, marker='o', linestyle='--')
plt.title('Energía Acumulada por Número de Factores')
plt.xlabel('Número de Factores')
plt.ylabel('Energía Acumulada')
plt.axhline(y=0.9, color='r', linestyle='--')  # Línea en 80% de energía
plt.grid(True)
plt.show()


In [26]:
# Determinar el número mínimo de factores que explican el 90% de la varianza
num_factors = np.argmax(explained_variance_cumsum >= 0.9) + 1
print(f"Número de factores necesarios para cubrir el 80% de la energía: {num_factors}")


Número de factores necesarios para cubrir el 80% de la energía: 8


## 5. SVD con enmascaramiento del 20% de Ítems


Para evaluar el modelo, dividimos los datos en conjuntos de entrenamiento y prueba, ocultando el 20% de las interacciones de cada usuario. Esto permite evaluar la capacidad predictiva del modelo en datos no vistos.



In [ ]:
data_filtered_grouped = data_filtered.groupby(['user_id', 'artist_name']).size().reset_index(name='listens')
data_filtered_grouped.head()

In [63]:
trainset_scaled = data_filtered_grouped
trainset_scaled['listens'] = data_filtered_grouped.listens/np.percentile(data_filtered_grouped['listens'], 95)

In [64]:
from sklearn.model_selection import train_test_split
# Ocultar el 20% de las escuchas por usuario en el conjunto de prueba
trainset, testset = train_test_split(data_filtered_grouped, test_size = 0.2, stratify = data_filtered_grouped.user_id)


Ahora que tenemos los conjuntos de entrenamiento y prueba divididos, configuraremos el conjunto de datos en el formato requerido por Surprise. Esto implica cargar los datos con un rango de escucha adecuado para Surprise y construir los objetos trainset y testset.

In [65]:

# Definir el rango de puntuaciones con base en el máximo número de escuchas

max_listens = trainset['listens'].max()
reader = Reader(rating_scale=(0, max_listens))

# Cargar el conjunto de entrenamiento en formato Surprise
trainset_s = Dataset.load_from_df(trainset[['user_id', 'artist_name', 'listens']], reader).build_full_trainset()

# Preparar el conjunto de prueba en formato de lista de tuplas (user_id, artist_name, listens)
testset_s = list(testset[['user_id', 'artist_name', 'listens']].itertuples(index=False, name=None))


## 6. Entrenamiento del Modelo SVD

Entrenamos el modelo SVD utilizando el conjunto de entrenamiento. Para la cantidad de factores latentes (n_factors), utilizamos el valor obtenido previamente en el análisis de energía acumulada.

In [66]:
# Crear y entrenar el modelo SVD con el número de factores seleccionado
svd_model = SVD(n_factors=num_factors)
svd_model.fit(trainset_s)


## 7. Generación de Recomendaciones


Utilizamos el modelo entrenado para generar recomendaciones para cada usuario. Para cada usuario, se seleccionarán canciones no escuchadas para hacer predicciones de recomendación.

In [ ]:
trainset.head()

In [68]:
# Función que aplica `predict` del modelo a cada fila
def get_predictions(row):
    prediccion = svd_model.predict(row['user_id'], row['artist_name']).est
    return prediccion


In [ ]:
trainset['listens_reconstruida'] = trainset.apply(get_predictions, axis=1)
trainset.head()

**Cálculo del RMSE**

Evaluamos la precisión del modelo usando el RMSE, calculando la diferencia promedio entre las listens y listens_reconstruida.

In [ ]:
# Calcular el RMSE entre listens y listens_reconstruida
rmse = np.sqrt(((trainset['listens'] - trainset['listens_reconstruida']) ** 2).mean())

# Mostrar el resultado
print(f"RMSE: {rmse}")

In [ ]:
testset['listens_reconstruida'] = testset.apply(get_predictions, axis=1)
testset.head()

In [ ]:
# Calcular el RMSE entre listens y listens_reconstruida
rmse = np.sqrt(((testset['listens'] - testset['listens_reconstruida']) ** 2).mean())

# Mostrar el resultado
print(f"RMSE: {rmse}")

**Cálculo del Hit Rate**

Para evaluar las predicciones de escucha/no escucha, usamos un umbral de 0.2. Si listens o listens_reconstruida son mayores a 0.2, los asignamos a 1 (escuchado), de lo contrario a 0 (no escuchado), y calculamos el porcentaje de aciertos (Hit Rate).

In [ ]:
# Crear las versiones binarizadas de listens y listens_reconstruida con el umbral de 0.2
testset['listens_bin'] = (testset['listens'] >= 0.2).astype(int)
testset['listens_reconstruida_bin'] = (testset['listens_reconstruida'] >= 0.2).astype(int)

# Calcular el Hit Rate: la proporción de coincidencias entre listens_bin y listens_reconstruida_bin
hit_rate = (testset['listens_bin'] == testset['listens_reconstruida_bin']).mean()

# Mostrar el resultado
print(f"Hit Rate: {hit_rate}")


In [ ]:
# Calcular el Hit Rate por usuario
hit_rate_per_user = testset.groupby('user_id').apply(
    lambda df: (df['listens_bin'] == df['listens_reconstruida_bin']).mean()
)

# Calcular el promedio de Hit Rate de todos los usuarios
average_hit_rate = hit_rate_per_user.mean()

# Mostrar el resultado
print(f"Hit Rate promedio por usuario: {average_hit_rate}")